<a href="https://colab.research.google.com/github/NeerajMehta15/Agentic-AI/blob/main/Travel_Agent_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- We are building a Travel Agent. This is V1 without and tool integration

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_groq

In [ ]:
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [ ]:
import os
import json
from typing import Dict, List, Any, Annotated
from typing_extensions import TypedDict

import langgraph as lg
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_groq import ChatGroq

In [ ]:
from google.colab import userdata
Groq_key = userdata.get('Groq_key_v2')
Tavily_key = userdata.get('Tavily')

In [ ]:
#Initializing LLM that can be used for this agent, here we are using gpt-3.5 turbo.
#Defining chatbot function and how it can input the message
llm = ChatGroq(model_name="llama-3.3-70b-versatile",  groq_api_key=Groq_key)

In [ ]:
# Define dictionary templates (for documentation purposes)
CONSTRAINTS_TEMPLATE = {
    "destination": "Barcelona, Spain",
    "start_date": "2025-04-15",
    "end_date": "2025-04-20",
    "budget": 2000.0,
    "interests": ["architecture", "food", "beach"],
    "accommodation_preference": "mid-range hotel",
    "pace": "moderate",
    "dietary_restrictions": ["vegetarian"],
    "transport_preference": "public transportation"
}

ACTIVITY_TEMPLATE = {
    "name": "Sagrada Familia Tour",
    "description": "Guided tour of Gaudí's masterpiece",
    "location": "Sagrada Familia, Barcelona",
    "duration_hours": 2.5,
    "cost": 25.0,
    "category": "architecture",
    "suitable_for_interests": ["architecture"]
}

DAY_PLAN_TEMPLATE = {
    "date": "2025-04-15",
    "activities": [ACTIVITY_TEMPLATE],
    "accommodation": "Hotel Barcelona Central",
    "total_day_cost": 150.0,
    "transportation": ["metro", "walking"]
}

ITINERARY_TEMPLATE = {
    "destination": "Barcelona, Spain",
    "days": [DAY_PLAN_TEMPLATE],
    "total_cost": 1850.0,
    "remaining_budget": 150.0
}

In [ ]:
#Helper function to parse JSON from LLM output
def parse_json_from_llm(text):
    """Extract and parse JSON from LLM output text"""
    try:
        # Try to parse the entire text as JSON first
        return json.loads(text)
    except json.JSONDecodeError:
        # If that fails, try to extract JSON between triple backticks
        import re
        json_match = re.search(r'```json\n(.*?)\n```', text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except json.JSONDecodeError:
                pass

        # Try to extract any JSON-like structure
        json_match = re.search(r'({.*})', text, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group(1))
            except json.JSONDecodeError:
                pass

        # If all else fails, return the original text
        raise ValueError(f"Could not parse JSON from LLM output: {text}")


In [ ]:
# 1. Constraint Modeling Agent
constraint_modeling_template = ChatPromptTemplate.from_messages([
    ("system", """You are a Travel Constraint Specialist. Your job is to:
    1. Extract user travel preferences and constraints from their input
    2. Apply common-sense constraints (e.g., travel times, logical schedules, local customs)
    3. Create a structured representation of all constraints

    Return your response as a JSON object with these keys:
    - destination: string (where they want to go)
    - start_date: string in YYYY-MM-DD format
    - end_date: string in YYYY-MM-DD format
    - budget: number (total budget in USD)
    - interests: array of strings
    - accommodation_preference: string
    - pace: string (relaxed, moderate, busy)
    - dietary_restrictions: array of strings or null
    - transport_preference: string or null

    Ensure you capture all essential details and add reasonable defaults for missing information.
    Return ONLY valid JSON without any other text or explanation.
    """),
    MessagesPlaceholder(variable_name="chat_history"),("human", "{user_input}"),
])

# Function to create constraint modeling node
def constraint_modeling_node():
    @lg.node
    def process_constraints(state: Dict[str, Any]) -> Dict[str, Any]:
        chat_history = state.get("chat_history", [])
        user_input = state.get("user_input", "")

        try:
            response = llm.invoke(constraint_modeling_template.format_messages(chat_history=chat_history,user_input=user_input))
            constraints = parse_json_from_llm(response.content)
            return {"constraints": constraints, "status": "success"}
        except Exception as e:
            return {"error": str(e), "status": "error"}

    return process_constraints


In [ ]:
# 2. Planning Agent
planning_template = ChatPromptTemplate.from_messages([
    ("system", """You are a Travel Itinerary Planner. Your job is to:
    1. Create a day-by-day travel plan based on user constraints
    2. Ensure the plan is logistically feasible (considering travel times, opening hours)
    3. Structure activities for each day within the given time frame
    4. Balance the pace according to user preferences
    5. Stay within budget constraints

    Return a complete and well-organized travel itinerary as a JSON object with these keys:
    - destination: string
    - days: array of day objects, each containing:
      - date: string in YYYY-MM-DD format
      - activities: array of activity objects, each containing:
        - name: string
        - description: string
        - location: string
        - duration_hours: number
        - cost: number
        - category: string
        - suitable_for_interests: array of strings
      - accommodation: string
      - total_day_cost: number
      - transportation: array of strings
    - total_cost: number
    - remaining_budget: number

    Return ONLY valid JSON without any other text or explanation.
    """),
    ("human", """
    Create a detailed travel itinerary based on these constraints:
    {constraints_json}

    Make sure to balance activities each day according to the preferred pace,
    and ensure the total cost stays within budget.
    """),
])

# Function to create planning node
def planning_node():
    @lg.node
    def create_plan(state: Dict[str, Any]) -> Dict[str, Any]:
        constraints = state.get("constraints", {})

        try:
            # Convert constraints to JSON string for the prompt
            constraints_json = json.dumps(constraints)
            response = llm.invoke(planning_template.format_messages(constraints_json=constraints_json))
            itinerary = parse_json_from_llm(response.content)
            return {"itinerary": itinerary, "status": "success"}
        except Exception as e:
            return {"error": str(e), "status": "error"}

    return create_plan


In [ ]:
 #3. Recommendation Agent
recommendation_template = ChatPromptTemplate.from_messages([
    ("system", """You are a Travel Recommendation Specialist. Your job is to:
    1. Enhance an existing travel itinerary with personalized recommendations
    2. Suggest specific restaurants, attractions, and activities based on user interests
    3. Add local insights that match the user's preferences
    4. Ensure all recommendations maintain budget constraints

    You will receive an itinerary and should return an ENHANCED version of that itinerary
    with the same JSON structure but with added or improved activities and details.

    The JSON structure should have these keys:
    - destination: string
    - days: array of day objects, each containing:
      - date: string in YYYY-MM-DD format
      - activities: array of activity objects, each containing:
        - name: string
        - description: string
        - location: string
        - duration_hours: number
        - cost: number
        - category: string
        - suitable_for_interests: array of strings
      - accommodation: string
      - total_day_cost: number
      - transportation: array of strings
    - total_cost: number
    - remaining_budget: number

    Return ONLY valid JSON without any other text or explanation.
    """),
    ("human", """
    Enhance this travel itinerary with personalized recommendations:
    {itinerary_json}

    User constraints and preferences:
    {constraints_json}

    Focus on providing specific, personalized recommendations that match their interests.
    """),
])

# Function to create recommendation node
def recommendation_node():
    @lg.node
    def enhance_with_recommendations(state: Dict[str, Any]) -> Dict[str, Any]:
        constraints = state.get("constraints", {})
        itinerary = state.get("itinerary", {})

        try:
            # Convert to JSON strings for the prompt
            constraints_json = json.dumps(constraints)
            itinerary_json = json.dumps(itinerary)

            # Get enhanced recommendations
            response = llm.invoke(recommendation_template.format_messages(constraints_json=constraints_json,itinerary_json=itinerary_json))

            enhanced_itinerary = parse_json_from_llm(response.content)
            return {"enhanced_itinerary": enhanced_itinerary, "status": "success"}
        except Exception as e:
            return {"error": str(e), "status": "error"}

    return enhance_with_recommendations


In [ ]:
# 4. Response formatting node
def formatting_node():
    @lg.node
    def format_response(state: Dict[str, Any]) -> Dict[str, Any]:
        enhanced_itinerary = state.get("enhanced_itinerary", {})
        constraints = state.get("constraints", {})

        try:
            # Create a user-friendly response
            response_template = ChatPromptTemplate.from_messages([
                ("system", """You are a helpful Travel Assistant. Format the travel itinerary
                in a clear, well-structured, and engaging way for the user. Include:
                - A summary of the trip
                - Daily breakdown with activities
                - Cost information
                - Any special recommendations

                Make it visually appealing and easy to read.
                """),
                ("human", """
                Please format this travel itinerary for the user:
                {itinerary_json}

                Based on these user preferences:
                {constraints_json}
                """),
            ])

            response = llm.invoke(response_template.format_messages(itinerary_json=json.dumps(enhanced_itinerary),constraints_json=json.dumps(constraints)))

            return {"formatted_response": response.content, "status": "success"}
        except Exception as e:
            return {"error": str(e), "status": "error"}

    return format_response

In [ ]:
# Creating the graph
def create_travel_agent_graph():
    # Initialize the graph (without 'name' argument)
    workflow = StateGraph(input=str, output=str)

    # Add nodes
    workflow.add_node("constraint_modeling", constraint_modeling_node())
    workflow.add_node("planning", planning_node())
    workflow.add_node("recommendation", recommendation_node())
    workflow.add_node("format_response", formatting_node())

    # Add edges
    workflow.add_edge("constraint_modeling", "planning")
    workflow.add_edge("planning", "recommendation")
    workflow.add_edge("recommendation", "format_response")
    workflow.add_edge("format_response", END)

    # Compile the graph
    app = workflow.compile()
    return app


In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
# Function to run the travel planning system
def run_travel_planner(user_input: str):
    travel_agent = create_travel_agent_graph()
    result = travel_agent.invoke({"user_input": user_input})

    if "formatted_response" in result:
        return result["formatted_response"]
    elif "error" in result:
        return f"Error: {result['error']}"
    else:
        return "Something went wrong with the travel planning system."

In [ ]:
if __name__ == "__main__":
    user_query = """
    I want to plan a trip to Barcelona, Spain for 5 days from April 15 to April 20, 2025.
    My budget is $2000. I'm interested in architecture, food, and beach activities.
    I prefer a moderate pace of travel and would like to stay in a mid-range hotel.
    I'm vegetarian and prefer to use public transportation.
    """

    response = run_travel_planner(user_query)
    print(response)